In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from data import InMemoryExerciseData
from lstm import LSTM

def initialize_sess():
    global sess
    ruv = set(sess.run(tf.report_uninitialized_variables()))
    uv = [v for v in tf.global_variables() if v.name.split(':')[0].encode('ascii') in ruv]
    tf.variables_initializer(uv).run()
    
def reset_sess():
    global sess
    tf.reset_default_graph()
    sess.close()
    sess = tf.InteractiveSession()    

sess = tf.InteractiveSession()

min_seq_length = 5
max_seq_length = 3000
min_correct = 2
min_responses_for_skill = 16800

In [2]:
df = pd.read_csv('data/bridge_to_algebra_2006_2007_train.txt', sep='\t', lineterminator='\r',
                 usecols=['Anon Student Id', 'KC(SubSkills)', 'Correct First Attempt', 'Corrects',
                          'Incorrects', 'Step Start Time','Problem Name', 'Step Name'])


In [3]:
# DATETIME IS STORED IN NANOSECONDS
df['date'] = pd.to_datetime(df['Step Start Time']).astype(np.int64) // 10 ** 9
df.drop(['Step Start Time'], axis=1, inplace=True)

In [4]:
df['_count'] = 1
grouped_df = df.groupby('KC(SubSkills)').sum()

In [5]:
for idx, x in enumerate(grouped_df['_count'].sort_values(ascending=False)):
    if x < min_responses_for_skill:
        break
chosen_skill_names = grouped_df['_count'].sort_values(ascending=False)[0:idx].index
print("Num KCs: %d" % (len(chosen_skill_names)))
df = df[df['KC(SubSkills)'].isin(chosen_skill_names)]

Num KCs: 12


In [6]:
grouped_df = df.groupby('Anon Student Id').sum()
filtered_uids = grouped_df[(grouped_df['_count'] >= min_seq_length) &
                           (grouped_df['_count'] <= max_seq_length) &
                           (grouped_df['Corrects'] >= min_correct)].reset_index()['Anon Student Id']
print("Num Students: %d" % (len(filtered_uids)))
df = df[df['Anon Student Id'].isin(filtered_uids)]

Num Students: 1100


In [7]:
def new_day(df, new_day_threshold=5, verbose=True):
    """
    Processes data to include correctness of the previous question and whether it has been a day since the
    the last response to any skill.
    Ex. For exercises A and B:
        [0, 0, 1, 1, 0, 1]
        Means that the current response is for exercise B, is incorrect and it has been a day since the 
        previous response to an exercise.
    """
    num_students = df['Anon Student Id'].nunique()
    num_skills = df['KC(SubSkills)'].nunique()
    new_day_threshold = 5 * 60 * 60 # Convert to seconds

    corrects = np.zeros((num_students, max_seq_length, num_skills))
    mask = np.zeros((num_students, max_seq_length, num_skills))
    sequences = np.zeros((num_students, max_seq_length, 3 * num_skills))

    exercise_index_map = {}
    for idx, exercise in enumerate(df['KC(SubSkills)'].unique()):
        exercise_index_map[exercise] = idx * 3

    for row_idx, uid in enumerate(df['Anon Student Id'].unique()):
        uid_df = df[df['Anon Student Id'] == uid]
        col_idx = 0
        prev_date = 0.0

        for _, event in uid_df.iterrows():
            idx = exercise_index_map[event['KC(SubSkills)']]
            sequences[row_idx, col_idx, idx] = 1
            sequences[row_idx, col_idx, idx + 1] = event['Correct First Attempt']
            if event['date'] - prev_date > new_day_threshold:
                sequences[row_idx, col_idx] += np.array([0, 0, 1] * num_skills)
                        
            prev_date = event['date']
            corrects[row_idx, col_idx, idx // 3] = event['Correct First Attempt']
            mask[row_idx, col_idx, idx // 3] = 1
            col_idx += 1

        if verbose and row_idx % 100 == 0:
            print("Processed %d" % row_idx)

    return InMemoryExerciseData(sequences, mask, corrects)

def correctness_only(df, verbose=True):
    """
    Processes data to only include correctness of the previous question as input.
    Ex. For exercises A and B:
        [0, 0, 1, 0]
        Means that the current response is for exercise B and is incorrect.
    """
    num_students = df['Anon Student Id'].nunique()
    num_skills = df['KC(SubSkills)'].nunique()

    corrects = np.zeros((num_students, max_seq_length, num_skills))
    mask = np.zeros((num_students, max_seq_length, num_skills))
    sequences = np.zeros((num_students, max_seq_length, 2 * num_skills))

    exercise_index_map = {}
    for idx, exercise in enumerate(df['KC(SubSkills)'].unique()):
        exercise_index_map[exercise] = idx * 2

    for row_idx, uid in enumerate(df['Anon Student Id'].unique()):
        uid_df = df[df['Anon Student Id'] == uid]
        col_idx = 0

        for _, event in uid_df.iterrows():
            idx = exercise_index_map[event['KC(SubSkills)']]
            sequences[row_idx, col_idx, idx] = 1
            sequences[row_idx, col_idx, idx + 1] = event['Correct First Attempt']

            corrects[row_idx, col_idx, idx // 2] = event['Correct First Attempt']
            mask[row_idx, col_idx, idx // 2] = 1
            col_idx += 1

        if verbose and row_idx % 100 == 0:
            print("Processed %d" % row_idx)

    return InMemoryExerciseData(sequences, mask, corrects)

def new_day_by_exercise(df, new_day_threshold=5, verbose=True):
    """
    Processes data to include correctness of the previous question and whether it has been a day since the
    the last response to this skill.
    Ex. For exercises A and B:
        [0, 0, 1, 1, 0, 0]
        Means that the current response is for exercise B, is incorrect and it is not a new day for B. It
        is a new day for A.
    """
    num_students = df['Anon Student Id'].nunique()
    num_skills = df['KC(SubSkills)'].nunique()
    new_day_threshold = 5 * 60 * 60 # Convert to seconds

    corrects = np.zeros((num_students, max_seq_length, num_skills))
    mask = np.zeros((num_students, max_seq_length, num_skills))
    sequences = np.zeros((num_students, max_seq_length, 3 * num_skills))

    exercise_index_map = {}
    for idx, exercise in enumerate(df['KC(SubSkills)'].unique()):
        exercise_index_map[exercise] = idx * 3

    for row_idx, uid in enumerate(df['Anon Student Id'].unique()):
        uid_df = df[df['Anon Student Id'] == uid]
        col_idx = 0
        prev_dates = np.zeros((3 * num_skills))

        for _, event in uid_df.iterrows():
            idx = exercise_index_map[event['KC(SubSkills)']]
            sequences[row_idx, col_idx, idx] = 1
            sequences[row_idx, col_idx, idx + 1] = event['Correct First Attempt']
            sequences[row_idx, col_idx] += ((np.ones((3 * num_skills)) * event['date'] - prev_dates) * (prev_dates != 0)) > new_day_threshold
                        
            prev_dates[idx + 2] = event['date']
            corrects[row_idx, col_idx, idx // 3] = event['Correct First Attempt']
            mask[row_idx, col_idx, idx // 3] = 1
            col_idx += 1

        if verbose and row_idx % 100 == 0:
            print("Processed %d" % row_idx)

    return InMemoryExerciseData(sequences, mask, corrects)

In [8]:
correctness_only_data = correctness_only(df)
new_day_data = new_day(df)
new_day_by_exercise_data = new_day_by_exercise(df)

datasets = [correctness_only_data, new_day_data, new_day_by_exercise_data]

Processed 0
Processed 100
Processed 200
Processed 300
Processed 400
Processed 500
Processed 600
Processed 700
Processed 800
Processed 900
Processed 1000
Processed 0
Processed 100
Processed 200
Processed 300
Processed 400
Processed 500
Processed 600
Processed 700
Processed 800
Processed 900
Processed 1000
Processed 0
Processed 100
Processed 200
Processed 300
Processed 400
Processed 500
Processed 600
Processed 700
Processed 800
Processed 900
Processed 1000


In [9]:
print("--UNWEIGHTED--")
for data, name in zip(datasets, ["Correctness Only", "New Day Aggregate", "New Day by Exercise"]):
    print(name)
    reset_sess()
    lstm = LSTM(hidden_dim=200,
                output_dim=data.targets.shape[2],
                input_dim=data.inputs.shape[2],
                learning_rate=1e-2,
                batch_size=64,
                num_layers=1)

    lstm.build_model(tf.sigmoid)

    k = 15
    avg_acc = 0.0
    avg_auc = 0.0
    avg_mae = 0.0
    for fold in data.k_fold(k):
        print("Fold %d" % fold)
        tf.global_variables_initializer().run()
        lstm.train(sess, data, epochs=3)
        acc, baseline, auc, mae = lstm.test(sess, data)
        avg_acc += acc / k
        avg_auc += auc / k
        avg_mae += mae / k
    
    print("Average Accuracy: %.8f, Average AUC %.8f, Average Mean Absolute Error %.8f" % (avg_acc, avg_auc, avg_mae))

--UNWEIGHTED--
Correctness Only
Fold 1
Accuracy: 0.88614, Baseline: 0.88614, AUC: 0.98451, MAE: 0.14899
Fold 2
Accuracy: 0.90523, Baseline: 0.90535, AUC: 0.98750, MAE: 0.17492
Fold 3
Accuracy: 0.88632, Baseline: 0.88632, AUC: 0.98004, MAE: 0.16075
Fold 4
Accuracy: 0.89472, Baseline: 0.89472, AUC: 0.97538, MAE: 0.14934
Fold 5
Accuracy: 0.87740, Baseline: 0.87740, AUC: 0.98139, MAE: 0.17583
Fold 6
Accuracy: 0.90195, Baseline: 0.90195, AUC: 0.97973, MAE: 0.16864
Fold 7
Accuracy: 0.87234, Baseline: 0.87234, AUC: 0.98545, MAE: 0.22059
Fold 8
Accuracy: 0.87243, Baseline: 0.87243, AUC: 0.98630, MAE: 0.19669
Fold 9
Accuracy: 0.89203, Baseline: 0.89203, AUC: 0.98387, MAE: 0.22111
Fold 10
Accuracy: 0.91574, Baseline: 0.91574, AUC: 0.99001, MAE: 0.19883
Fold 11
Accuracy: 0.86302, Baseline: 0.86302, AUC: 0.98428, MAE: 0.22652
Fold 12
Accuracy: 0.87025, Baseline: 0.87025, AUC: 0.98743, MAE: 0.19593
Fold 13
Accuracy: 0.87208, Baseline: 0.87208, AUC: 0.97799, MAE: 0.19591
Fold 14
Accuracy: 0.90591, B

In [10]:
print("--WEIGHTED 1.5x--")
for data, name in zip(datasets, ["Correctness Only", "New Day Aggregate", "New Day by Exercise"]):
    print(name)
    data.target_masks = data.target_masks * 3/2 - data.targets * (3/2 - 1)
    reset_sess()
    lstm = LSTM(hidden_dim=200,
                output_dim=data.targets.shape[2],
                input_dim=data.inputs.shape[2],
                learning_rate=1e-2,
                batch_size=64,
                num_layers=1)

    lstm.build_model(tf.sigmoid)

    k = 15
    avg_acc = 0.0
    avg_auc = 0.0
    avg_mae = 0.0
    for fold in data.k_fold(k):
        print("Fold %d" % fold)
        tf.global_variables_initializer().run()
        lstm.train(sess, data, epochs=3)
        acc, baseline, auc, mae = lstm.test(sess, data)
        avg_acc += acc / k
        avg_auc += auc / k
        avg_mae += mae / k
    
    print("Average Accuracy: %.8f, Average AUC %.8f, Average Mean Absolute Error %.8f" % (avg_acc, avg_auc, avg_mae))

--WEIGHTED 1.5x--
Correctness Only
Fold 1
Accuracy: 0.88808, Baseline: 0.88826, AUC: 0.98387, MAE: 0.21906
Fold 2
Accuracy: 0.90399, Baseline: 0.90404, AUC: 0.98733, MAE: 0.18826
Fold 3
Accuracy: 0.88078, Baseline: 0.88078, AUC: 0.97666, MAE: 0.22670
Fold 4
Accuracy: 0.89136, Baseline: 0.89136, AUC: 0.97496, MAE: 0.20171
Fold 5
Accuracy: 0.88830, Baseline: 0.88731, AUC: 0.98122, MAE: 0.22500
Fold 6
Accuracy: 0.90122, Baseline: 0.90122, AUC: 0.98119, MAE: 0.22881
Fold 7
Accuracy: 0.87024, Baseline: 0.87024, AUC: 0.98475, MAE: 0.23530
Fold 8
Accuracy: 0.87680, Baseline: 0.87680, AUC: 0.98508, MAE: 0.26000
Fold 9
Accuracy: 0.89189, Baseline: 0.89189, AUC: 0.98295, MAE: 0.27784
Fold 10
Accuracy: 0.90663, Baseline: 0.90663, AUC: 0.98955, MAE: 0.22358
Fold 11
Accuracy: 0.86592, Baseline: 0.86592, AUC: 0.98470, MAE: 0.24329
Fold 12
Accuracy: 0.87474, Baseline: 0.87474, AUC: 0.98872, MAE: 0.18619
Fold 13
Accuracy: 0.86853, Baseline: 0.86853, AUC: 0.97378, MAE: 0.21723
Fold 14
Accuracy: 0.90631

In [22]:
reset_sess()